In [ ]:
import sys
from pathlib import Path
sys.path.extend([str(Path('.').resolve()), str(Path('..').resolve())])
import pandas as pd
import src.agents.dependency_agent as dep
import src.agents.orchestrator as orch
import src.tools.security as sec


In [ ]:
from pathlib import Path
SAMPLES_DIR = (Path.cwd() / '..' / 'samples').resolve()
samples = {
    'vulns': str(SAMPLES_DIR / 'requirements_vulns.txt'),
    'bad': str(SAMPLES_DIR / 'requirements_bad.txt')
}
py_version = '3.11'


In [ ]:
scan_v = dep.run_scan(samples['vulns'], py_version, 'data')
scan_b = dep.run_scan(samples['bad'], py_version, 'data')
print('vulns count:', scan_v['count'])
print('bad count:', scan_b['count'])


In [ ]:
df_v = pd.DataFrame(scan_v['packages'])
df_b = pd.DataFrame(scan_b['packages'])
df_v.head()


In [ ]:
query1 = 'Scan my requirements.txt for vulns; propose minimal safe pins for Python 3.11; enforce no pre-releases; show first fixed versions.'
out1 = orch.advise(query1, 'llama', samples['vulns'], history=[], prompt_style='deliberate_json', planning_mode='consensus-3', verify=True)
print(out1['answer_text'])


In [ ]:
pd.DataFrame(out1['meta']['tool_calls'])


In [ ]:
query2 = 'Is urllib3==1.25.8 vulnerable? List CVEs and first fixed version.'
out2 = orch.advise(query2, 'phi', samples['vulns'], history=[], prompt_style='strict_schema', planning_mode='single', verify=True)
print(out2['answer_text'])


In [ ]:
sec_data = sec.run_pip_audit(samples['vulns'])
pd.DataFrame(sec_data.get('vulnerabilities') or [])


In [ ]:
pd.DataFrame([{
    'provider': out1['meta']['provider'],
    'llm': out1['meta']['llm'],
    'prompt_style': out1['meta']['prompt_style'],
    'planning_mode': out1['meta']['planning_mode'],
    'verify_ms': out1['meta']['verify_ms'],
    'summ_cache_hit': out1['meta'].get('summ_cache_hit'),
    'verify_cache_hit': out1['meta'].get('verify_cache_hit')
}])
